In [44]:
import pyarrow.fs
from ray.data import ActorPoolStrategy
import sycamore
from sycamore.functions.tokenizer import HuggingFaceTokenizer
from sycamore.llms import OpenAIModels, OpenAI
from sycamore.transforms import COALESCE_WHITESPACE
from sycamore.transforms.merge_elements import MarkedMerger
from sycamore.transforms.partition import SycamorePartitioner
from sycamore.transforms.extract_schema import OpenAISchemaExtractor, OpenAIPropertyExtractor
from sycamore.transforms.embed import SentenceTransformerEmbedder

In [45]:
from sycamore.data.document import Document
from dateutil import parser
def convert_timestamp(doc: Document) -> Document:
    if "dateAndTime" not in doc.properties['entity'] and "dateTime" not in doc.properties['entity']:
        return doc
    raw_date: str = doc.properties['entity'].get('dateAndTime') or doc.properties['entity'].get('dateTime')
    raw_date = raw_date.replace("Local", "")
    parsed_date = parser.parse(raw_date, fuzzy=True)
    extracted_date = parsed_date.date()
    doc.properties['entity']['day'] = extracted_date.day
    doc.properties['entity']['month'] = extracted_date.month
    doc.properties['entity']['year'] = extracted_date.year
    if parsed_date.utcoffset():
        doc.properties['entity']['dateTime'] = parsed_date.isoformat()
    else:
        doc.properties['entity']['dateTime'] = parsed_date.isoformat() + "Z"
    
    return doc

In [46]:
paths = ["s3://aryn-public/ntsb/"]
fsys = pyarrow.fs.S3FileSystem(region="us-east-1", anonymous=True)

llm = OpenAI(OpenAIModels.GPT_3_5_TURBO.value)
tokenizer = HuggingFaceTokenizer("thenlper/gte-small")

ctx = sycamore.init()

ds = (
    ctx.read.binary(paths, binary_format="pdf", filesystem=fsys)
    # Parition with the sycamore partitioner, pulling out tables and images. ActorPoolStrategy(size=3) works best on my particular hardware
    # but your mileage may vary depending on your RAM.
    .partition(partitioner=SycamorePartitioner(extract_table_structure=True, extract_images=True), compute=ActorPoolStrategy(size=3))
    # Get rid of spurious whitespace charaters
    .regex_replace(COALESCE_WHITESPACE)
    # Automatically determine a schema of additional metadata to extract from documents
    .extract_batch_schema(schema_extractor=OpenAISchemaExtractor("FlightAccidentReport", llm=llm, num_of_elements=35))
    # Extract the metadata specified by that schema
    .extract_properties(property_extractor=OpenAIPropertyExtractor(llm=llm, num_of_elements=35))
    # Merge elements into larger chunks
    .mark_bbox_preset(tokenizer=tokenizer)
    .merge(merger=MarkedMerger())
    # Convert extracted timestamps to better-structured form using the function above
    .map(convert_timestamp)
    # Copy document properties to each document's sub-elements
    .spread_properties(["path", "entity"])
    # Split elements that are too big to embed
    .split_elements(tokenizer=tokenizer, max_tokens=512)
    # Convert all Elements to Documents
    .explode()
    # Generate a series of hashes to represent each document. For use with near-duplicate detection
    .sketch()
    # Embed each document
    .embed(embedder=SentenceTransformerEmbedder(model_name="sentence-transformers/all-MiniLM-L6-v2", batch_size=100))
)

In [47]:
# Write to a persistent duckdb database
# - into a specific database (as specified by url) 
# - into a specific table (as specified by table name)
persistent_db = "demo.db"
persistent_table = "demo_table"
ds.write.duckdb(
    db_url=persistent_db,
    table_name=persistent_table
)

2024-07-02 14:11:45,761	WARNING util.py:560 -- The argument ``compute`` is deprecated in Ray 2.9. Please specify argument ``concurrency`` instead. For more information, see https://docs.ray.io/en/master/data/transforming-data.html#stateful-transforms.
2024-07-02 14:11:45,762	WARNING util.py:560 -- The argument ``compute`` is deprecated in Ray 2.9. Please specify argument ``concurrency`` instead. For more information, see https://docs.ray.io/en/master/data/transforming-data.html#stateful-transforms.
2024-07-02 14:11:45,768	WARNING util.py:560 -- The argument ``compute`` is deprecated in Ray 2.9. Please specify argument ``concurrency`` instead. For more information, see https://docs.ray.io/en/master/data/transforming-data.html#stateful-transforms.
2024-07-02 14:11:45,774	INFO streaming_executor.py:112 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-07-01_12-05-17_251742_53329/logs/ray-data
2024-07-02 14:11:45,774	INFO streaming_executor.py:113 -- Execution plan o

- ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap) 1:   0%|          | 0/…

- MapBatches(regex_replace)->MapBatches(BaseMapTransformCustom__Extract) 2:   0%|          | 0/65 [00:00<?, ?i…

- MapBatches(extract_properties)->MapBatches(sort_by_page_bbox)->MapBatches(mark_drop_tiny)->MapBatches(mark_d…

Running 0:   0%|          | 0/65 [00:00<?, ?it/s]

2024-07-02 14:11:56,466	ERROR streaming_executor_state.py:449 -- An exception was raised from a task of operator "ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap)". Dataset execution will now abort. To ignore this exception and continue, set DataContext.max_errored_blocks.
2024-07-02 14:11:56,474	WARNING actor_pool_map_operator.py:292 -- To ensure full parallelization across an actor pool of size 3, the Dataset should consist of at least 3 distinct blocks. Consider increasing the parallelism when creating the Dataset.
2024-07-02 14:11:56,477	WARNING actor_pool_map_operator.py:292 -- To ensure full parallelization across an actor pool of size 1, the Dataset should consist of at least 1 distinct blocks. Consider increasing the parallelism when creating the Dataset.
2024-07-02 14:11:56,480	WARNING actor_pool_map_operator.py:292 -- To ensure full parallelization across an actor pool of size 1, the Dataset should consist of at least 1 distinct blocks. Co

RayTaskError(UserCodeException): [36mray::ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap)()[39m (pid=70168, ip=127.0.0.1, actor_id=db1f3d3cbef323e4f1ea395d01000000, repr=MapWorker(ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap)))
AttributeError: 'SycamorePartitioner' object has no attribute '_use_cache'

The above exception was the direct cause of the following exception:

[36mray::ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap)()[39m (pid=70168, ip=127.0.0.1, actor_id=db1f3d3cbef323e4f1ea395d01000000, repr=MapWorker(ReadBinary->Map(BinaryScan._to_document)->MapBatches(BaseMapTransformCallable___wrap)))
  File "/Users/karansampath/Library/Caches/pypoetry/virtualenvs/sycamore-ai--zTjaFUY-py3.10/lib/python3.10/site-packages/ray/data/_internal/planner/plan_udf_map_op.py", line 110, in fn
    return ray.data._cached_fn(item, *fn_args, **fn_kwargs)
  File "/Users/karansampat

In [5]:
# We connect to the DuckDB to perform operations
import duckdb
data_conn = duckdb.connect("demo.db")

In [38]:
# DuckDB is not production ready yet for efficient Vector Similarity Search on persistent databases (https://duckdb.org/docs/extensions/vss.html), 
# so we load our data into an in-memory database for this demo. In future versions, once DuckDB productionizes this feature, there will be no more need for this code cell

# Load from disk into Pandas Dataframe as an intermediate step
df = data_conn.execute("SELECT * FROM demo_table").fetchdf()
df.dropna(subset=["embeddings"],inplace=True)

# Now, we load the data into an in-memory database. Notice that we specify beforehand since we need the embedding column (must be specified as a FLOAT[N] where N is specified) 
# for Vector Similarity Search using HNSW to work
schema = {
            "doc_id": "VARCHAR",
            "embeddings": "FLOAT[384]",
            "properties": "MAP(VARCHAR, VARCHAR)",
            "text_representation": "VARCHAR",
            "bbox": "DOUBLE[]",
            "shingles": "BIGINT[]",
            "type": "VARCHAR",
        }
in_memory_db = duckdb.connect(":default:")
# in_memory_db.install_extension("vss")
# in_memory_db.load_extension("vss")
in_memory_db.execute(f"""CREATE TABLE in_memory_table (doc_id {schema.get('doc_id')},
                      embeddings {schema.get('embeddings')}, properties {schema.get('properties')}, 
                      text_representation {schema.get('text_representation')}, bbox {schema.get('bbox')}, 
                      shingles {schema.get('shingles')}, type {schema.get('type')})"""
    )
in_memory_db.execute("""INSERT INTO in_memory_table SELECT * FROM df; 
                      CREATE INDEX in_memory_table_index ON in_memory_table USING HNSW(embeddings)""")

ConnectionException: Connection Error: Connection has already been closed

In [18]:
# For queries, let's define an embedding function for the question that helps us easily compare the two vectors and generate optimal results // Run NN search
from sentence_transformers import SentenceTransformer
minilm = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

class Embedder():
    def __init__(self, llm):
        self.llm = llm
    
    def embed_query(self, question):
        v = self.llm.encode(question).tolist()
        return v

embedder = Embedder(minilm)

/Users/karansampath/Library/Caches/pypoetry/virtualenvs/sycamore-ai--zTjaFUY-py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Let us now see the performance of the ANN HNSW search using DuckDB // array_distance is the nn lookup
q = "What automobile type is the most accident prone?"
in_memory_db.sql(f"SELECT doc_id, text_representation, properties FROM in_memory_table ORDER BY array_distance(embeddings, {embedder.embed_query(q)}::FLOAT[384]) LIMIT 10;")
# Pretty print tables
# text representation --> print one cell out (of the top result). 

┌──────────────────────┬──────────────────────┬────────────────────────────────────────────────────────────────────────┐
│        doc_id        │ text_representation  │                               properties                               │
│       varchar        │       varchar        │                         map(varchar, varchar)                          │
├──────────────────────┼──────────────────────┼────────────────────────────────────────────────────────────────────────┤
│ 73f87ba9-ff4e-4d75…  │ The National Trans…  │ {score=0.8915606737136841, page_numbers=[1], page_number=1, path=s3:…  │
│ 06b31736-b0ed-4c57…  │ The National Trans…  │ {score=0.8774372339248657, page_numbers=[1], page_number=1, path=s3:…  │
│ d701d8a7-b5fa-4a8f…  │ The National Trans…  │ {score=0.7743222713470459, page_numbers=[1], page_number=1, path=s3:…  │
│ 36e85e87-85b7-4261…  │ The National Trans…  │ {score=0.7248827219009399, page_numbers=[1], page_number=1, path=s3:…  │
│ 0918cf95-4cbb-452c…  │ The Nat

In [66]:
# Let us now see the performance of the ANN HNSW search using DuckDB 
q = "Traffic Statistics"
in_memory_db.sql(f"SELECT doc_id, text_representation, properties FROM in_memory_table ORDER BY array_distance(embeddings, {embedder.embed_query(q)}::FLOAT[384]) LIMIT 10;")

┌──────────────────────┬──────────────────────┬────────────────────────────────────────────────────────────────────────┐
│        doc_id        │ text_representation  │                               properties                               │
│       varchar        │       varchar        │                         map(varchar, varchar)                          │
├──────────────────────┼──────────────────────┼────────────────────────────────────────────────────────────────────────┤
│ 3c82fabc-9f99-4dfe…  │ Airport Informatio…  │ {score=0.7310920357704163, page_numbers=[5], page_number=5, path=s3:…  │
│ 8f3d0214-9de3-48c3…  │ Airport Informatio…  │ {score=0.702387809753418, page_numbers=[5], page_number=5, path=s3:/…  │
│ 1c6083df-00a8-484e…  │ Airport Informatio…  │ {score=0.6993858218193054, page_numbers=[5], page_number=5, path=s3:…  │
│ b10638a0-2ea2-4d23…  │ Airport Informatio…  │ {score=0.7263772487640381, page_numbers=[5], page_number=5, path=s3:…  │
│ c0594fed-b67a-44b9…  │ Airport

In [67]:
# We can also see the performance of the DuckDB query and check the index is being used by using the in-built visualization below 
q = "What automobile type is the most accident prone?"
in_memory_db.sql(f"EXPLAIN SELECT doc_id, text_representation, properties FROM in_memory_table ORDER BY array_distance(embeddings, {embedder.embed_query(q)}::FLOAT[384]) LIMIT 10")

┌───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  explain_key  │                                            explain_value                                             │
│    varchar    │                                               varchar                                                │
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ physical_plan │ ┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│    …  │
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [10]:
# Finally we can build an index with other distance metrics, the default is Euclidean Distance ('l2sq'), with Cosine similarity ('cosine', 'array_cosine_simarlity') 
# and Inner Product ('ip', 'array_inner_product') also supported
in_memory_db.execute("CREATE INDEX in_memory_ip_index ON in_memory_table USING HNSW(embeddings) WITH (metric = 'ip')")
in_memory_db.sql(f"SELECT doc_id, text_representation, properties FROM in_memory_table ORDER BY array_inner_product(embeddings, {embedder.embed_query(q)}::FLOAT[384]) LIMIT 10;")

┌──────────────────────┬──────────────────────┬────────────────────────────────────────────────────────────────────────┐
│        doc_id        │ text_representation  │                               properties                               │
│       varchar        │       varchar        │                         map(varchar, varchar)                          │
├──────────────────────┼──────────────────────┼────────────────────────────────────────────────────────────────────────┤
│ 73f87ba9-ff4e-4d75…  │ The National Trans…  │ {score=0.8915606737136841, page_numbers=[1], page_number=1, path=s3:…  │
│ 06b31736-b0ed-4c57…  │ The National Trans…  │ {score=0.8774372339248657, page_numbers=[1], page_number=1, path=s3:…  │
│ d701d8a7-b5fa-4a8f…  │ The National Trans…  │ {score=0.7743222713470459, page_numbers=[1], page_number=1, path=s3:…  │
│ 36e85e87-85b7-4261…  │ The National Trans…  │ {score=0.7248827219009399, page_numbers=[1], page_number=1, path=s3:…  │
│ 0918cf95-4cbb-452c…  │ The Nat

In [27]:
in_memory_db.sql(f"ALTER TABLE in_memory_table ADD COLUMN metadata VARCHAR[]")

CatalogException: Catalog Error: Column with name metadata already exists!

In [28]:
# conn = duckdb.connect(database=':memory:',
#     config={
#         # Sample configuration to restrict some DuckDB capabilities
#         # List is not exhaustive. Please review DuckDB documentation.
#             "enable_external_access": "false",
#             "autoinstall_known_extensions": "false",
#             "autoload_known_extensions": "false"
#         }
# )
from langchain_community.vectorstores.duckdb import DuckDB
from langchain_openai import OpenAIEmbeddings
vector_store = DuckDB(connection=in_memory_db, embedding=embedder, id_key= "doc_id", text_key="text_representation", vector_key="embeddings", table_name="in_memory_table")
# vector_store.add_texts(['text1', 'text2'])
result = vector_store.similarity_search('traffic')

TypeError: the JSON object must be str, bytes or bytearray, not float

In [39]:
from langchain_openai import ChatOpenAI  
from langchain.chains import RetrievalQAWithSourcesChain  
import os
# completion llm  
llm = ChatOpenAI(  
    openai_api_key=os.environ.get("OPENAI_API_KEY"),  
    model_name='gpt-3.5-turbo',  
    temperature=0.0  
)  
qa = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vector_store.as_retriever()  
)  
qa.run(q)

/Users/karansampath/Library/Caches/pypoetry/virtualenvs/sycamore-ai--zTjaFUY-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


ValueError: `run` not supported when there is not exactly one output key. Got ['answer', 'sources'].

In [ ]:
# OPTIONAL: If you would like to remove your database after running the code above
try:
    os.unlink(persistent_db)
except Exception as e:
    print(f"Error deleting {persistent_db}: {e}")